# 02 - Data Cleaning

**Customer Lifetime Value Prediction**

**Team:** The Starks
- Othmane Zizi (261255341)
- Fares Joni (261254593)
- Tanmay Giri (261272443)

This notebook cleans and preprocesses the Online Retail II dataset.

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path('../src').resolve()))
from data_loader import load_raw_data, clean_data

pd.set_option('display.max_columns', None)

## 1. Load Raw Data

In [2]:
# Load the raw data
data_path = Path('../data/raw/online_retail_II.xlsx')
df = load_raw_data(data_path)

print(f"Raw data shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Raw data shape: (1067371, 8)

Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 2. Data Quality Check

In [3]:
print("Data Quality Summary Before Cleaning:")
print(f"\nTotal rows: {len(df):,}")

# Missing values
print("\nMissing Values:")
for col in df.columns:
    missing = df[col].isnull().sum()
    if missing > 0:
        print(f"  {col}: {missing:,} ({missing/len(df)*100:.2f}%)")

# Cancelled transactions
cancelled = df['Invoice'].astype(str).str.startswith('C').sum()
print(f"\nCancelled transactions (Invoice starts with 'C'): {cancelled:,} ({cancelled/len(df)*100:.2f}%)")

# Negative/zero values
neg_qty = (df['Quantity'] <= 0).sum()
neg_price = (df['Price'] <= 0).sum()
print(f"\nNon-positive Quantity: {neg_qty:,} ({neg_qty/len(df)*100:.2f}%)")
print(f"Non-positive Price: {neg_price:,} ({neg_price/len(df)*100:.2f}%)")

Data Quality Summary Before Cleaning:

Total rows: 1,067,371

Missing Values:
  Description: 4,382 (0.41%)
  Customer ID: 243,007 (22.77%)



Cancelled transactions (Invoice starts with 'C'): 19,494 (1.83%)

Non-positive Quantity: 22,950 (2.15%)
Non-positive Price: 6,207 (0.58%)


## 3. Clean the Data

In [4]:
# Apply cleaning function
df_clean = clean_data(df)

print(f"Original rows: {len(df):,}")
print(f"Cleaned rows: {len(df_clean):,}")
print(f"Rows removed: {len(df) - len(df_clean):,} ({(len(df) - len(df_clean))/len(df)*100:.1f}%)")

Original rows: 1,067,371
Cleaned rows: 805,549
Rows removed: 261,822 (24.5%)


In [5]:
# Verify cleaning
print("\nPost-Cleaning Verification:")
print(f"  Missing Customer ID: {df_clean['Customer ID'].isnull().sum()}")
print(f"  Cancelled transactions: {df_clean['Invoice'].str.startswith('C').sum()}")
print(f"  Non-positive Quantity: {(df_clean['Quantity'] <= 0).sum()}")
print(f"  Non-positive Price: {(df_clean['Price'] <= 0).sum()}")


Post-Cleaning Verification:
  Missing Customer ID: 0
  Cancelled transactions: 0
  Non-positive Quantity: 0
  Non-positive Price: 0


In [6]:
# Data info after cleaning
print("\nData Types:")
print(df_clean.dtypes)
print(f"\nMemory usage: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Data Types:
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID             int64
Country                object
TotalAmount           float64
dtype: object

Memory usage: 227.44 MB


## 4. Handle Outliers

In [7]:
# Check for extreme outliers
print("Outlier Analysis:")
print("\nQuantity Statistics:")
print(df_clean['Quantity'].describe())
print(f"\n99th percentile: {df_clean['Quantity'].quantile(0.99)}")
print(f"Max: {df_clean['Quantity'].max()}")

print("\nPrice Statistics:")
print(df_clean['Price'].describe())
print(f"\n99th percentile: {df_clean['Price'].quantile(0.99)}")
print(f"Max: {df_clean['Price'].max()}")

print("\nTotalAmount Statistics:")
print(df_clean['TotalAmount'].describe())
print(f"\n99th percentile: {df_clean['TotalAmount'].quantile(0.99)}")
print(f"Max: {df_clean['TotalAmount'].max()}")

Outlier Analysis:

Quantity Statistics:
count    805549.000000
mean         13.290522
std         143.634088
min           1.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64

99th percentile: 128.0
Max: 80995

Price Statistics:
count    805549.000000
mean          3.206561
std          29.199173
min           0.001000
25%           1.250000
50%           1.950000
75%           3.750000
max       10953.500000
Name: Price, dtype: float64

99th percentile: 14.95
Max: 10953.5

TotalAmount Statistics:
count    805549.000000
mean         22.026505
std         224.041928
min           0.001000
25%           4.950000
50%          11.850000
75%          19.500000
max      168469.600000
Name: TotalAmount, dtype: float64

99th percentile: 201.60000000000002
Max: 168469.6


In [8]:
# Remove extreme outliers (optional - keeping for now but flagging)
# Very high prices might be bulk/wholesale orders which are valid
# We'll keep the data but be aware of outliers during modeling

# Flag potential outliers for reference
qty_threshold = df_clean['Quantity'].quantile(0.999)
price_threshold = df_clean['Price'].quantile(0.999)

extreme_qty = (df_clean['Quantity'] > qty_threshold).sum()
extreme_price = (df_clean['Price'] > price_threshold).sum()

print(f"Extreme quantity outliers (>99.9th percentile): {extreme_qty}")
print(f"Extreme price outliers (>99.9th percentile): {extreme_price}")

Extreme quantity outliers (>99.9th percentile): 766
Extreme price outliers (>99.9th percentile): 775


## 5. Summary Statistics of Cleaned Data

In [9]:
print("Cleaned Dataset Summary:")
print(f"\nRows: {len(df_clean):,}")
print(f"Unique Customers: {df_clean['Customer ID'].nunique():,}")
print(f"Unique Invoices: {df_clean['Invoice'].nunique():,}")
print(f"Unique Products: {df_clean['StockCode'].nunique():,}")
print(f"Countries: {df_clean['Country'].nunique()}")
print(f"\nDate Range: {df_clean['InvoiceDate'].min().date()} to {df_clean['InvoiceDate'].max().date()}")
print(f"Duration: {(df_clean['InvoiceDate'].max() - df_clean['InvoiceDate'].min()).days} days")
print(f"\nTotal Revenue: £{df_clean['TotalAmount'].sum():,.2f}")

Cleaned Dataset Summary:

Rows: 805,549
Unique Customers: 5,878
Unique Invoices: 36,969
Unique Products: 4,631
Countries: 41

Date Range: 2009-12-01 to 2011-12-09
Duration: 738 days

Total Revenue: £17,743,429.18


In [10]:
# Customer-level summary
customer_summary = df_clean.groupby('Customer ID').agg({
    'Invoice': 'nunique',
    'TotalAmount': 'sum',
    'InvoiceDate': ['min', 'max']
}).reset_index()

customer_summary.columns = ['Customer ID', 'NumOrders', 'TotalSpend', 'FirstPurchase', 'LastPurchase']

print("\nCustomer-Level Statistics:")
print(customer_summary[['NumOrders', 'TotalSpend']].describe())


Customer-Level Statistics:
         NumOrders     TotalSpend
count  5878.000000    5878.000000
mean      6.289384    3018.616737
std      13.009406   14737.731040
min       1.000000       2.950000
25%       1.000000     348.762500
50%       3.000000     898.915000
75%       7.000000    2307.090000
max     398.000000  608821.650000


## 6. Save Cleaned Data

In [11]:
# Save cleaned data to processed folder
output_path = Path('../data/processed/cleaned_retail.csv')
df_clean.to_csv(output_path, index=False)

print(f"Cleaned data saved to: {output_path}")
print(f"File size: {output_path.stat().st_size / 1024**2:.2f} MB")

Cleaned data saved to: ../data/processed/cleaned_retail.csv
File size: 73.79 MB


In [12]:
# Verify saved data
df_verify = pd.read_csv(output_path, parse_dates=['InvoiceDate'])
print(f"\nVerification - Loaded shape: {df_verify.shape}")
df_verify.head()


Verification - Loaded shape: (805549, 9)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


## 7. Cleaning Summary

### Steps Performed:
1. Removed rows with missing Customer ID (~20% of data)
2. Removed cancelled transactions (Invoice starting with 'C')
3. Removed rows with non-positive Quantity
4. Removed rows with non-positive Price
5. Created TotalAmount = Quantity * Price

### Data Retained:
- ~75% of original data retained after cleaning
- All retained rows have valid customer IDs for CLV calculation

### Next Steps:
- Feature engineering (RFM + behavioral features)
- Create train/test temporal split